In [65]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**step 2:**

In [66]:
#step2:load dataset
import pandas as pd
path='/content/drive/MyDrive/Colab Notebooks/Preprocessed Fake Reviews Detection Dataset.csv/Preprocessed Fake Reviews Detection Dataset.csv'
df=pd.read_csv(path)
df.head()

,Unnamed: 0,category,rating,label,text_
0,0,Home_and_Kitchen_5,5,1,love well made sturdi comfort i love veri pretti
1,1,Home_and_Kitchen_5,5,1,love great upgrad origin i 've mine coupl year
2,2,Home_and_Kitchen_5,5,1,thi pillow save back i love look feel pillow
3,3,Home_and_Kitchen_5,1,1,miss inform use great product price i
4,4,Home_and_Kitchen_5,5,1,veri nice set good qualiti we set two month


**Step 3:**

In [67]:
#step 3: check missing values
df.isnull().sum()


,0
Unnamed: 0,0
category,0
rating,0
label,0
text_,1


In [68]:
df=df.dropna()
df.isnull().sum()

,0
Unnamed: 0,0
category,0
rating,0
label,0
text_,0


# Step 4:

In [69]:
#step 4: remove unuseless columns

df=df.drop(columns=['Unnamed: 0','category','rating'])
df.head()

,label,text_
0,1,love well made sturdi comfort i love veri pretti
1,1,love great upgrad origin i 've mine coupl year
2,1,thi pillow save back i love look feel pillow
3,1,miss inform use great product price i
4,1,veri nice set good qualiti we set two month


In [70]:
# rename column --->> text_ = text
df=df.rename(columns={'text_':'text'})
df

,label,text
0,1,love well made sturdi comfort i love veri pretti
1,1,love great upgrad origin i 've mine coupl year
2,1,thi pillow save back i love look feel pillow
3,1,miss inform use great product price i
4,1,veri nice set good qualiti we set two month
...,...,...
40427,0,i read review say bra ran small i order two ba...
40428,1,i n't sure exactli would it littl larg small s...
40429,0,you wear hood wear hood wear jacket without ho...
40430,1,i like noth dress the reason i gave star i ord...


# step 5:

In [71]:
# Convert text too lowercase
df['text']=df['text'].str.lower()
df.head()

,label,text
0,1,love well made sturdi comfort i love veri pretti
1,1,love great upgrad origin i 've mine coupl year
2,1,thi pillow save back i love look feel pillow
3,1,miss inform use great product price i
4,1,veri nice set good qualiti we set two month


#step 6

In [72]:
import re

# Check if any review still contains uppercase letters
has_uppercase = df['text'].str.contains(r'[A-Z]').any()

print(has_uppercase)

False


In [73]:
# Remove special character & numbers ,emojy
df['text'] = df['text'].apply(lambda x: re.sub('[^a-zA-Z ]', '', x))
df

,label,text
0,1,love well made sturdi comfort i love veri pretti
1,1,love great upgrad origin i ve mine coupl year
2,1,thi pillow save back i love look feel pillow
3,1,miss inform use great product price i
4,1,veri nice set good qualiti we set two month
...,...,...
40427,0,i read review say bra ran small i order two ba...
40428,1,i nt sure exactli would it littl larg small si...
40429,0,you wear hood wear hood wear jacket without ho...
40430,1,i like noth dress the reason i gave star i ord...


In [74]:
df

,label,text
0,1,love well made sturdi comfort i love veri pretti
1,1,love great upgrad origin i ve mine coupl year
2,1,thi pillow save back i love look feel pillow
3,1,miss inform use great product price i
4,1,veri nice set good qualiti we set two month
...,...,...
40427,0,i read review say bra ran small i order two ba...
40428,1,i nt sure exactli would it littl larg small si...
40429,0,you wear hood wear hood wear jacket without ho...
40430,1,i like noth dress the reason i gave star i ord...


#step7: Tokenization using nltk

In [75]:
import nltk
from nltk.tokenize import word_tokenize

# Download the 'punkt' tokenizer if not already present
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

# Apply word_tokenize to each text entry in the 'text' column
df['text'] = df['text'].apply(word_tokenize)


# Display the first few rows with the new tokenized_text column
display(df.head())

,label,text
0,1,"[love, well, made, sturdi, comfort, i, love, v..."
1,1,"[love, great, upgrad, origin, i, ve, mine, cou..."
2,1,"[thi, pillow, save, back, i, love, look, feel,..."
3,1,"[miss, inform, use, great, product, price, i]"
4,1,"[veri, nice, set, good, qualiti, we, set, two,..."


In [76]:
df

,label,text
0,1,"[love, well, made, sturdi, comfort, i, love, v..."
1,1,"[love, great, upgrad, origin, i, ve, mine, cou..."
2,1,"[thi, pillow, save, back, i, love, look, feel,..."
3,1,"[miss, inform, use, great, product, price, i]"
4,1,"[veri, nice, set, good, qualiti, we, set, two,..."
...,...,...
40427,0,"[i, read, review, say, bra, ran, small, i, ord..."
40428,1,"[i, nt, sure, exactli, would, it, littl, larg,..."
40429,0,"[you, wear, hood, wear, hood, wear, jacket, wi..."
40430,1,"[i, like, noth, dress, the, reason, i, gave, s..."


#step8: Remove stop word

In [77]:
import nltk
from nltk.corpus import stopwords

# Download the 'stopwords' corpus if not already present
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def remove_stopwords(text_tokens):
    return [word for word in text_tokens if word not in stop_words]

df['text'] = df['text'].apply(remove_stopwords)

display(df.head())

,label,text
0,1,"[love, well, made, sturdi, comfort, love, veri..."
1,1,"[love, great, upgrad, origin, mine, coupl, year]"
2,1,"[thi, pillow, save, back, love, look, feel, pi..."
3,1,"[miss, inform, use, great, product, price]"
4,1,"[veri, nice, set, good, qualiti, set, two, month]"


# step 9: lemmatization

In [78]:
import nltk
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

df['text'] = df['text'].apply(
    lambda x: [lemmatizer.lemmatize(word) for word in x]
)
display(df.head())

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,label,text
0,1,"[love, well, made, sturdi, comfort, love, veri..."
1,1,"[love, great, upgrad, origin, mine, coupl, year]"
2,1,"[thi, pillow, save, back, love, look, feel, pi..."
3,1,"[miss, inform, use, great, product, price]"
4,1,"[veri, nice, set, good, qualiti, set, two, month]"


#step 10 : join words

In [79]:
df['text'] = df['text'].apply(lambda x: ' '.join(x))

In [80]:
df.head()

,label,text
0,1,love well made sturdi comfort love veri pretti
1,1,love great upgrad origin mine coupl year
2,1,thi pillow save back love look feel pillow
3,1,miss inform use great product price
4,1,veri nice set good qualiti set two month
